In [40]:
import pandas as pd
import numpy as np
import pandas as pd
import itertools 
import matplotlib.pyplot as plt
from terminaltables import AsciiTable
from scipy.stats import chi2

In [41]:
#SE IMPORTA EL CSV
ford_data = pd.read_csv('../CSV/Ford3.csv')
ford_data

,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-04-01,4.63,4.77,4.36,4.40,3.736051,119174500
1,2020-04-02,4.39,4.57,4.25,4.36,3.702087,90891700
2,2020-04-03,4.40,4.42,4.17,4.24,3.600194,85659900
3,2020-04-06,4.54,4.67,4.41,4.53,3.846434,102205300
4,2020-04-07,4.85,5.03,4.63,4.71,3.999273,115150400
...,...,...,...,...,...,...,...
980,2024-02-23,12.10,12.26,12.09,12.14,12.140000,32152100
981,2024-02-26,12.13,12.27,11.95,11.95,11.950000,40241300
982,2024-02-27,12.02,12.17,11.96,12.00,12.000000,36508800
983,2024-02-28,11.97,12.40,11.96,12.30,12.300000,51447900


In [42]:
#SE BORRAN LAS COLUMNAS QUE NO SE USARAN
ford_data = ford_data.drop(columns=['Open', 'High', 'Low', 'Close','Volume'])
ford_data

,Date,Adj Close
0,2020-04-01,3.736051
1,2020-04-02,3.702087
2,2020-04-03,3.600194
3,2020-04-06,3.846434
4,2020-04-07,3.999273
...,...,...
980,2024-02-23,12.140000
981,2024-02-26,11.950000
982,2024-02-27,12.000000
983,2024-02-28,12.300000


In [43]:
percentage = [0,]
for i in range(1, len(ford_data)):
    percent = (ford_data['Adj Close'][i] - ford_data['Adj Close'][i-1]) / ford_data['Adj Close'][i-1]
    percentage.append(percent)
ford_data.insert(2, 'Change %', percentage)

std_dev = [0,] * 20
for i in range(21, len(percentage) + 1):
    standar_dev = np.std(percentage[i-20:i])
    std_dev.append(standar_dev)
ford_data.insert(3, 'std dev', std_dev)

last_stv_case = ford_data['std dev'][len(ford_data)-1]
print(last_stv_case)

states = [0,] * 20
for j in range(20, len(percentage)):
    change = percentage[j]
    desv = std_dev[j]
    if change >= last_stv_case:
        state = "S"
    elif change <= last_stv_case:
        state = "B"
    else:
        state = "N"
    states.append(state)
ford_data.insert(4, 'Current State', states)

next_state_column = []
for i in range(len(ford_data)-1):
    next_state_column.append(ford_data['Current State'][i+1])

previows_state_column = [0]
for i in range(1, len(ford_data)-1):
    previows_state_column.append(ford_data['Current State'][i-1])

ford_data = ford_data.drop(ford_data.index[-1])
ford_data.insert(5, 'Next State', next_state_column)
ford_data.insert(4, 'Previous State', previows_state_column)

complete_table = AsciiTable([ford_data.columns] + list(ford_data.values))
print("Complete Table \n", complete_table.table)


0.023854059535827227
Complete Table 
 +------------+-----------+------------------------+----------------------+----------------+---------------+------------+
| Date       | Adj Close | Change %               | std dev              | Previous State | Current State | Next State |
+------------+-----------+------------------------+----------------------+----------------+---------------+------------+
| 2020-04-01 | 3.736051  | 0.0                    | 0.0                  | 0              | 0             | 0          |
| 2020-04-02 | 3.702087  | -0.009090882324679096  | 0.0                  | 0              | 0             | 0          |
| 2020-04-03 | 3.600194  | -0.027523124118909146  | 0.0                  | 0              | 0             | 0          |
| 2020-04-06 | 3.846434  | 0.06839631419862369    | 0.0                  | 0              | 0             | 0          |
| 2020-04-07 | 3.999273  | 0.03973524568470437    | 0.0                  | 0              | 0             | 0      

In [44]:
#SE QUITAN LAS PRIMERAS 20 FILAS, PUES ESTAS NO CUENTAN CON ESTADO ACTUAL
ford_trim=ford_data.iloc[21:]
table_week_21 = AsciiTable([ford_trim.columns]+list(ford_trim.values))
print("Table since week 21 \n",table_week_21.table)

Table since week 21 
 +------------+-----------+------------------------+----------------------+----------------+---------------+------------+
| Date       | Adj Close | Change %               | std dev              | Previous State | Current State | Next State |
+------------+-----------+------------------------+----------------------+----------------+---------------+------------+
| 2020-05-01 | 4.177584  | -0.03339895213529503   | 0.040472763891518396 | B              | B             | B          |
| 2020-05-04 | 4.126638  | -0.012195086921053077  | 0.03995608107590788  | B              | B             | B          |
| 2020-05-05 | 4.220039  | 0.022633679038481222   | 0.037655086431775536 | B              | B             | B          |
| 2020-05-06 | 4.135129  | -0.020120667131275284  | 0.03717865556646528  | B              | B             | B          |
| 2020-05-07 | 4.135129  | 0.0                    | 0.03399871069865621  | B              | B             | S          |
| 2020-05-

In [45]:
def markov_model(data):
    #PROBABILIDAD OBSERVADA
    states=np.unique(data['Current State'])
    states_mapping = {}
    for i,state in enumerate(states):
        states_mapping[state]=i

    #PROBABILIDAD ESPERADA
    transition_matrix = np.zeros(((len(states_mapping)), len(states_mapping)))
    for i in range(len(data)):
        transition_matrix[states_mapping[data.iloc[i]['Current State']], states_mapping[data.iloc[i]['Next State']]] += 1
    for i in range(len(transition_matrix)):
        transition_matrix[i] = transition_matrix[i] / sum(transition_matrix[i])
    data_frame_expected = pd.DataFrame(transition_matrix, columns=list(states_mapping.keys()), index=list(states_mapping.keys()))
    print(f"Probabilidad Esperada\n{data_frame_expected}\n")

    return transition_matrix
#SE ENVIAN LAS COLUMNAS DE ESTADO ACTUAL Y SIGUIENTE
transition_matrix = markov_model(ford_trim[['Current State', 'Next State']])

Probabilidad Esperada
          B         S
B  0.833126  0.166874
S  0.837500  0.162500



In [46]:
def sendInfoToAnalize():
    return transition_matrix